In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from pathlib import Path

In [ ]:
dPath = Path("../docs/dumps")

In [ ]:
import pickle

In [ ]:
with open(dPath / "train_data.pkl", 'rb') as filename:
    train_data = pickle.load(filename)

In [ ]:
with open(dPath / "valid_data.pkl", 'rb') as filename:
    valid_data = pickle.load(filename)

In [ ]:
X_train = train_data.drop("Detected", axis=1)

In [ ]:
y_train = train_data.Detected

In [ ]:
X_valid = valid_data.drop("Detected", axis=1)

In [ ]:
y_valid = valid_data.Detected

In [ ]:
with open(dPath / "rf_exp_04_names.pkl", 'rb') as filename:
    names = pickle.load(filename)

In [ ]:
X_train = X_train[names] 

In [ ]:
X_valid = X_valid[names]

In [ ]:
X_train.head()

In [ ]:
from imblearn.over_sampling import ADASYN

In [ ]:
sm = ADASYN(random_state=42, n_jobs=-1,  n_neighbors=5)

In [ ]:
%time X_train, y_train = sm.fit_resample(X_train, y_train)

In [ ]:
from catboost import CatBoostClassifier, Pool, cv

In [ ]:
cb = CatBoostClassifier(
    custom_loss=['AUC'],
    random_seed=42,
    cat_features=[1],
    iterations=1000,
    verbose=10)

In [ ]:
cb.fit(X_train, y_train);

In [ ]:
cb.save_model(str(dPath / 'cb_exp_1.dump'))

In [ ]:
from sklearn.metrics import classification_report
from sklearn import metrics

In [ ]:
def conf_matr(m):
    y_pred = m.predict(X_train)=='True'
    print(classification_report(y_train, y_pred))

In [ ]:
conf_matr(cb)

In [ ]:
cb.get_all_params()